In [2]:
from bs4 import BeautifulSoup as bs
import requests
import re
from google.colab import drive
import numpy as np
import pandas as pd

In [ ]:
drive.mount("/content/drive")

**ASSIGNMENT I**: creating a new json file

In [3]:
url = "https://dummyjson.com/products"

In [4]:
data = requests.get(url).json()

In [5]:
products = data["products"]


In [6]:
# GET LAST 20 PRODUCTS
last_20Products = products[-20:]

In [7]:
# CREATE AGGREGATE JSON  

In [8]:
new_json = {}

In [ ]:
[new_json.setdefault(prod["category"],[{}]) for prod in products if prod["category"] not in list(new_json.keys())]

In [ ]:
new_json

In [ ]:
for category in list(new_json.keys()):
  for product in products:
    if product["category"] == category:
      try:
        new_json[category][0][product["brand"]].insert(0,product["title"])
        # ADD PRICE AND NUMBER OF ITEMS
        new_json[category][0][product["brand"]][-2] += product['price']
        new_json[category][0][product["brand"]][-1] += 1
        # MAKE AVERAGE PRICE
        new_json[category][0][product["brand"]][-2] = round((float(new_json[category][0][product["brand"]][-2]) / new_json[category][0][product["brand"]][-1]),1)

      except KeyError:
        new_json[category][0][product["brand"]] = [0,0]
        new_json[category][0][product["brand"]].insert(0,product["title"])
        # ADD PRICE AND NUMBER OF ITEMS
        new_json[category][0][product["brand"]][-2] += product['price']
        new_json[category][0][product["brand"]][-1] += 1
        # MAKE AVERAGE PRICE
        new_json[category][0][product["brand"]][-2] = round((float(new_json[category][0][product["brand"]][-2]) / new_json[category][0][product["brand"]][-1]),1)

In [ ]:
# REMOVE THE NUMBER REPESENT ITEMS IN A CATEGORY
for category in list(new_json.keys()):
  for brand in list(new_json[category][0].keys()):
    new_json[category][0][brand].pop()


In [ ]:
new_json

{'smartphones': [{'Apple': ['iPhone X', 'iPhone 9', 724.0],
   'Samsung': ['Samsung Universe 9', 1249.0],
   'OPPO': ['OPPOF19', 280.0],
   'Huawei': ['Huawei P30', 499.0]}],
 'laptops': [{'APPle': ['MacBook Pro', 1749.0],
   'Samsung': ['Samsung Galaxy Book', 1499.0],
   'Microsoft Surface': ['Microsoft Surface Laptop 4', 1499.0],
   'Infinix': ['Infinix INBOOK', 1099.0],
   'HP Pavilion': ['HP Pavilion 15-DK1056WM', 1099.0]}],
 'fragrances': [{'Impression of Acqua Di Gio': ['perfume Oil', 13.0],
   'Royal_Mirage': ['Brown Perfume', 40.0],
   'Fog Scent Xpressio': ['Fog Scent Xpressio Perfume', 13.0],
   'Al Munakh': ['Non-Alcoholic Concentrated Perfume Oil', 120.0],
   'Lord - Al-Rehab': ['Eau De Perfume Spray', 30.0]}],
 'skincare': [{"L'Oreal Paris": ['Hyaluronic Acid Serum', 19.0],
   'Hemani Tea': ['Tree Oil 30ml', 12.0],
   'Dermive': ['Oil Free Moisturizer 100ml', 40.0],
   'ROREC White Rice': ['Skin Beauty Serum.', 46.0],
   'Fair & Clear': ['Freckle Treatment Cream- 15gm', 70

**ASSIGNMENT II** : scrap igihe.com from January to present

In [ ]:
unsortedLinks = []

for month in range(1, 11):
  for day in range(1, 32):
    l = "http://archive.org/wayback/available?url=igihe.com&timestamp=2022{:02d}{:02d}".format(month, day)
    response = requests.get(l).json()
    if  response['archived_snapshots']:
      unsortedLinks.append(response['archived_snapshots']['closest']["url"])

In [ ]:
len(unsortedLinks)

307

In [ ]:
arr = np.array(unsortedLinks)
series = pd.Series(arr)
unsortedLinks = series.unique()

In [ ]:
serious_links = {}


In [ ]:
# link = "https://web.archive.org/web/20220615035616/https://www.igihe.com/amakuru/"
def process_links(link):
  page = requests.get(link)
  soup = bs(page.text, "html.parser")
  
  b = soup.body
  get_titles = [link+re.sub("amakuru/","", a['href']) for span in b.find_all("span", class_ = "homenews-title") for a in span.find_all("a")]
  
  # get only links whose date is in current year
  pattern = re.compile(r"\d{8,}?")
  found_date = pattern.search(link).group()
  if found_date[3] != str(2):
    pass
  else:
    file_title = f"igihe-{found_date[0:4]}-{found_date[4:6]}-{found_date[6:8]}"
    try:
      for title in get_titles:
        serious_links[file_title].append(title)
    except KeyError:
      serious_links[file_title] = []
      for title in get_titles:
        serious_links[file_title].append(title)

In [ ]:
for link in unsortedLinks:
  process_links(link)

In [ ]:
serious_links

In [ ]:
def writeToFile(paragraphs, file):
  with open(file, "a+") as f:
    for line in paragraphs:
      f.write(line)

In [ ]:
def get_page():
  count = 1
  for key in list(serious_links.keys()):
    for link in set(serious_links[key]):
      p = requests.get(link).content
      soup = bs(p, "html.parser")
      spans = soup.find_all("div", class_= "fulltext margintop10")
      paragraphs = [p.get_text() for span in spans for p in span.find_all("p")]
      writeToFile(paragraphs = paragraphs, file = f"/content/drive/MyDrive/NLP fellowship/week1 tasks/{key}.txt")
    print(f"file {count} created")
    count += 1

In [ ]:
# FINISH THE PROGRAM
get_page()

**ASSIGNMENT III** : extract text from 5 pdf documents provided and write them in a single file

In [ ]:
!pip install textract


In [ ]:
!pip install PyPDF2

In [ ]:
 import os
 import textract, PyPDF2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# GET THE PDF FILES 
os.chdir("/content/drive/MyDrive/NLP fellowship/files")
PDFs = [file for file in os.listdir() if "Hansard" in file]

In [ ]:
PDFs

In [ ]:
def remove_firstPage(pdfs):
  pages = {}
  temp = PyPDF2.PdfFileWriter()
  for pdf in pdfs:
    pages[pdf] = []

  for pdf in pdfs:
    reader = PyPDF2.PdfFileReader(pdf)
    allowed_pages = reader.getNumPages()
    for page_number in range(1, allowed_pages-1):
      # pages[pdf].append(reader.getPage(page_number))
      temp.add_page(reader.getPage(page_number))

  return temp


In [ ]:
pages = remove_firstPage(PDFs)

In [ ]:
# create template pdf file
with open("/content/temp.pdf", "wb") as f:
  pages.write(f)

In [ ]:
# EXTRACT EACH PDF
def extract(file, new_destination = "PDFsExtracted.txt"):
  text = textract.process(file).decode("utf-8")
  lines = text.split("\n")
  with open(new_destination, "a+") as f:
    for line in lines:
      if line.strip():
        f.write(line+"\n")
  with open(new_destination, "a+") as f:
    f.write("\n")
  
extract("/content/temp.pdf")

**END OF THE WEEK ASSIGNMENT** data cleaning 

In [ ]:
engKinyFile = "/content/drive/MyDrive/NLP fellowship/week1 tasks/Eng_Kin-Paralleldata (1).csv" 

In [ ]:
# READ THE FILE
df = pd.read_csv(engKinyFile)
kiny = df['Kinyarwanda']
eng = df["English"]

In [ ]:
kiny

In [ ]:
# NORMALIZING THE FILE
kiny = kiny.str.lower()
eng = eng.str.lower()

In [ ]:
# REMOVE THE WHITE SPACES
kiny = kiny.str.strip()
eng = eng.str.strip()

In [ ]:
print(kiny)

In [ ]:
kiny_list = list(kiny)
eng_list = list(eng)

In [ ]:
kiny_list

In [ ]:
# REMOVE URLs and EMAILS
pattern = re.compile("\w+@[^\s\.]{3,}|([a-z]+\.[\w]{3,})|([^\s]+(\.net|\.org|\.com))|www\..*|htt\.*|[^\s]+@[^\s]+")
found = [pattern.findall(line) for line in eng_list if pattern.findall(line)]
EngNo_urls = [pattern.sub("",line) for line in eng_list]
KinyNo_urls = [pattern.sub("",line) for line in kiny_list]

In [ ]:
# EngNo_urls
found
# KinyNo_urls

In [ ]:
# REMOVE PANCTUATIONS 
panc_pattern = re.compile("[^\w\s]+")
EngNo_punc = [panc_pattern.sub("", line) for line in EngNo_urls]
KinyNo_punc = [panc_pattern.sub("", line) for line in KinyNo_urls]

In [ ]:
KinyNo_punc

In [ ]:
# REMOVE NUMBERS
dig_pattern = re.compile("\d+")
EngNo_dig = [dig_pattern.sub("", line) for line in EngNo_punc]
KinyNo_dig = [dig_pattern.sub("", line) for line in KinyNo_punc]

In [ ]:
KinyNo_dig

In [ ]:
# remove english words in kinyarwanda
# eng_pattern = re.compile("\w+[b-df-hj-np-tv-xz]\b")
eng_pattern = re.compile(r"\w+[b-df-hj-np-tv-xz]\b")
pure_kiny = [eng_pattern.sub("",line) for line in KinyNo_dig ]

In [ ]:
pure_kiny

In [ ]:
# OUTPUT DATA
df['Kinyarwanda'] = pd.DataFrame(pure_kiny)
df["English"] = pd.DataFrame(EngNo_dig)

In [ ]:
df.head(30)